In [1]:
!ls /media/hdd1

Anaconda3-5.0.0.1-Linux-x86_64.sh  MSongsDB
bazel-build-examples		   nvidia-digits
chainerrl			   nvidia-docker
cpp-ethereum			    programs
crawler				   pytorch
cuda_exaample			   RoboND-CNN-Lab
dataset				   RoboND-DeepLearning-Project
DeepLearning_Presentation.tar.gz   RoboND-Python-StarterKit
faiss				   RoboND-Segmentation-Lab
fastText-0.1.0			   sd_readfile.py
hive-testbench			   spring-boot
hyperledge-composer		   tensorflow
JSAnimation			   tf-sq2seq
lost+found			   tpcds
merged_file.txt			   v0.1.0.zip


In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
mta_data = pd.read_csv("/media/hdd1/merged_file.txt")

In [4]:
mta_data["DATE TIME"] = mta_data["DATE"] + " " + mta_data["TIME"]

In [9]:
mta_data["DATE TIME"].head()
mta_data["DATETIME2"] = pd.to_datetime(mta_data["DATE TIME"])

In [10]:
mta_data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE TIME,DATETIME,DATETIME2
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,03:00:00,REGULAR,6463109,2187558,12/30/2017 03:00:00,12/30/2017 03:00:00,2017-12-30 03:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,07:00:00,REGULAR,6463119,2187567,12/30/2017 07:00:00,12/30/2017 07:00:00,2017-12-30 07:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,11:00:00,REGULAR,6463178,2187654,12/30/2017 11:00:00,12/30/2017 11:00:00,2017-12-30 11:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,15:00:00,REGULAR,6463373,2187736,12/30/2017 15:00:00,12/30/2017 15:00:00,2017-12-30 15:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,19:00:00,REGULAR,6463712,2187803,12/30/2017 19:00:00,12/30/2017 19:00:00,2017-12-30 19:00:00


In [11]:
# Clean up the data
mta_cleaned_data = mta_data[(mta_data.EXITS != 0) | (mta_data.ENTRIES != 0)]

In [100]:
len(mta_cleaned_data)

10344462

In [13]:
mta_cleaned_data.to_pickle("mta_datetime.pkl")

In [17]:
#Lets Clean the Data
#Step 1 we are going to remove all the bad dates data which are not actual 4 hour window
# There are datasets which are poorly aggregated
valid_time_list = []
for i in mta_cleaned_data["TIME"].unique():
    if (i.split(":")[1] == "00" and i.split(":")[2] =="00"):
        valid_time_list.append(i)

mta_cleaned_data_filtered_valida_date = mta_cleaned_data.loc[mta_cleaned_data["TIME"].isin(valid_time_list)]       

In [18]:
#Now lets check the rows of the files
# number of rows before was 10344462
len(mta_cleaned_data_filtered_valida_date)

9439309

In [27]:
# So total data that we cleaned is 9439309
# So the percentage change
change_percent = ((10344462.0 - 9439309.0))
print (change_percent/10344462) * 100

8.75012156263


In [28]:
# Percentatge of files dropped in 8.7%
# Which is insignificant

In [30]:
# Lets add the total for all the Entries and Exits
#This will give us a gross idea of the top 20 stations
#The turnstile data always increment. So we would have to do a diff between the rows
# We are going to create a total column and add the entries and exits
mta_cleaned_data["TOTAL"] = mta_cleaned_data["ENTRIES"] + mta_cleaned_data["EXITS"]

/home/sdutta/.local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
# Lets explore the new dataframe
mta_cleaned_data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE TIME,DATETIME,DATETIME2,TOTAL
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,03:00:00,REGULAR,6463109,2187558,12/30/2017 03:00:00,12/30/2017 03:00:00,2017-12-30 03:00:00,8650667
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,07:00:00,REGULAR,6463119,2187567,12/30/2017 07:00:00,12/30/2017 07:00:00,2017-12-30 07:00:00,8650686
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,11:00:00,REGULAR,6463178,2187654,12/30/2017 11:00:00,12/30/2017 11:00:00,2017-12-30 11:00:00,8650832
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,15:00:00,REGULAR,6463373,2187736,12/30/2017 15:00:00,12/30/2017 15:00:00,2017-12-30 15:00:00,8651109
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/30/2017,19:00:00,REGULAR,6463712,2187803,12/30/2017 19:00:00,12/30/2017 19:00:00,2017-12-30 19:00:00,8651515


In [32]:
#The above records looks good.
#The next step would be lets groupby the stations
mta_total_by_station = mta_cleaned_data.groupby([['STATION']], as_index=False)[["TOTAL"]].agg('sum')

In [34]:
#Lets sort by total
mta_total_by_station_sorted = mta_total_by_station.sort_values(by="TOTAL", ascending=False)

In [37]:
#Lets get the the top 20 stations
mta_top_20_stations = mta_total_by_station_sorted.head(20)

In [99]:
#Lets create a list variable of the data
top_20_station_list = list(mta_top_20_stations["STATION"])
top_20_station_list

['42 ST-PORT AUTH',
 '23 ST',
 '34 ST-HERALD SQ',
 'CANAL ST',
 'TIMES SQ-42 ST',
 'CHAMBERS ST',
 'FULTON ST',
 '104 ST',
 '72 ST',
 '14 ST',
 '125 ST',
 '34 ST-PENN STA',
 '59 ST',
 'HIGH ST',
 '59 ST COLUMBUS',
 '3 AV-149 ST',
 'LEXINGTON AV/53',
 '57 ST-7 AV',
 '183 ST',
 'DEKALB AV']

In [154]:
#Data frame for 42 ST-Port AUTH
mta_42_street = mta_cleaned_data[mta_cleaned_data["STATION"]==top_20_station_list[0]]

In [155]:
mta_42_street2 = mta_cleaned_data[mta_cleaned_data["STATION"]==top_20_station_list[0]]

In [156]:
mta_42_street2_sorted = mta_42_street2.sort_values(by=["C/A","UNIT","SCP","DATETIME2"])

In [157]:
#mta_42_street2_sorted= mta_42_street2_sorted.drop(['DATETIME', 'ACTUAL_ENTRIES','ACTUAL_EXITS','ACTUAL_TOTAL'], axis=1)

In [123]:
mta_42_street2_sorted.to_pickle("42ndStreet_sorted.pkl")

In [174]:
mta_42_street2_sorted.reset_index()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE TIME,DATETIME,DATETIME2,TOTAL,Total_entries_exits,MONTH,YEAR
0,49002,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,05:00:00,REGULAR,10139410,15336325,12/30/2017 05:00:00,NaN,2017-12-30 05:00:00,25475735,114.0,12,2017
1,49003,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,09:00:00,REGULAR,10139450,15336583,12/30/2017 09:00:00,NaN,2017-12-30 09:00:00,25476033,298.0,12,2017
2,49004,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,13:00:00,REGULAR,10139590,15337199,12/30/2017 13:00:00,NaN,2017-12-30 13:00:00,25476789,756.0,12,2017
3,49005,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,17:00:00,REGULAR,10139935,15338050,12/30/2017 17:00:00,NaN,2017-12-30 17:00:00,25477985,1196.0,12,2017
4,49006,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,21:00:00,REGULAR,10140289,15338876,12/30/2017 21:00:00,NaN,2017-12-30 21:00:00,25479165,1180.0,12,2017
5,49007,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,01:00:00,REGULAR,10140768,15339158,12/31/2017 01:00:00,NaN,2017-12-31 01:00:00,25479926,761.0,12,2017
6,49008,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,05:00:00,REGULAR,10140852,15339204,12/31/2017 05:00:00,NaN,2017-12-31 05:00:00,25480056,130.0,12,2017
7,49009,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,09:00:00,REGULAR,10140903,15339385,12/31/2017 09:00:00,NaN,2017-12-31 09:00:00,25480288,232.0,12,2017
8,49010,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,13:00:00,REGULAR,10141098,15339793,12/31/2017 13:00:00,NaN,2017-12-31 13:00:00,25480891,603.0,12,2017
9,49011,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,17:00:00,REGULAR,10141284,15340443,12/31/2017 17:00:00,NaN,2017-12-31 17:00:00,25481727,836.0,12,2017


In [160]:
#Lets diff the columns as turnstile increases the number of rows
#mta_42_street2_sorted["Total_entries_exits"] = mta_42_street2_sorted["TOTAL"].diff()

In [161]:
#Clean the 42nd Street data which is not null. The first row
#mta_42_street2_sorted = mta_42_street2_sorted[mta_42_street2_sorted.Total_entries_exits.notnull()]

C/A
UNIT
SCP
STATION
LINENAME
DIVISION
DATE
TIME
DESC
ENTRIES
EXITS
DATE TIME
DATETIME
DATETIME2
TOTAL
Total_entries_exits
MONTH
YEAR


In [163]:
#mta_42_street2_sorted_non_negetive = mta_42_street2_sorted[(mta_42_street2_sorted["Total_entries_exits"]<90000) & (mta_42_street2_sorted["Total_entries_exits"]>0)]

In [164]:
#mta_42_street2_sorted["ACTUAL_TOTAL2"] = mta_42_street2_sorted["ACTUAL_ENTRIES"] + mta_42_street2_sorted["ACTUAL_EXITS"]

In [165]:
len(mta_42_street2_sorted)

112705

In [166]:
mta_42_street2_sorted.to_pickle("42ndStreet_non_neg.pkl")

In [167]:
mta_42_street2_sorted["MONTH"]=mta_42_street2_sorted["DATETIME2"].dt.month

/home/sdutta/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [168]:
mta_42_street2_sorted["YEAR"]=mta_42_street2_sorted["DATETIME2"].dt.year

/home/sdutta/.local/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [173]:
#mta_42_street2_sorted.groupby(["YEAR","MONTH"])["Total_entries_exits"].agg(sum)

for index, data in mta_42_street2_sorted:
    

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE TIME,DATETIME,DATETIME2,TOTAL,Total_entries_exits,MONTH,YEAR
49002,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,05:00:00,REGULAR,10139410,15336325,12/30/2017 05:00:00,NaN,2017-12-30 05:00:00,25475735,114.0,12,2017
49003,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,09:00:00,REGULAR,10139450,15336583,12/30/2017 09:00:00,NaN,2017-12-30 09:00:00,25476033,298.0,12,2017
49004,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,13:00:00,REGULAR,10139590,15337199,12/30/2017 13:00:00,NaN,2017-12-30 13:00:00,25476789,756.0,12,2017
49005,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,17:00:00,REGULAR,10139935,15338050,12/30/2017 17:00:00,NaN,2017-12-30 17:00:00,25477985,1196.0,12,2017
49006,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/30/2017,21:00:00,REGULAR,10140289,15338876,12/30/2017 21:00:00,NaN,2017-12-30 21:00:00,25479165,1180.0,12,2017
49007,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,01:00:00,REGULAR,10140768,15339158,12/31/2017 01:00:00,NaN,2017-12-31 01:00:00,25479926,761.0,12,2017
49008,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,05:00:00,REGULAR,10140852,15339204,12/31/2017 05:00:00,NaN,2017-12-31 05:00:00,25480056,130.0,12,2017
49009,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,09:00:00,REGULAR,10140903,15339385,12/31/2017 09:00:00,NaN,2017-12-31 09:00:00,25480288,232.0,12,2017
49010,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,13:00:00,REGULAR,10141098,15339793,12/31/2017 13:00:00,NaN,2017-12-31 13:00:00,25480891,603.0,12,2017
49011,N060,R010,01-00-00,42 ST-PORT AUTH,ACENGRS1237W,IND,12/31/2017,17:00:00,REGULAR,10141284,15340443,12/31/2017 17:00:00,NaN,2017-12-31 17:00:00,25481727,836.0,12,2017


In [153]:
!pwd

/home/sdutta/Tensorflow-starter/code/machinelearning/scratch
